In [2]:
!pip install faiss-cpu sentence-transformers transformers torch numpy pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.2 MB/s eta 0:00:00


In [3]:
import faiss
print(faiss.__version__)  # Should print the version

1.10.0


In [4]:
documents = [
    "RAG combines retrieval and generation for better answers.",
    "Vector databases store embeddings for fast search.",
    "Open-source models are free and powerful."
]

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Free, lightweight
embeddings = model.encode(documents, show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
import numpy as np
print(embeddings.shape)  # Should be (3, 384) for 3 docs

(3, 384)


In [7]:
import faiss

dimension = embeddings.shape[1]  # 384 for MiniLM
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity
index.add(embeddings)  # Add embeddings

In [8]:
query = "What is RAG?"
query_embedding = model.encode([query])
distances, indices = index.search(query_embedding, k=2)  # Top 2 matches
retrieved_docs = [documents[i] for i in indices[0]]
print("Retrieved:", retrieved_docs)
print("Distances:", distances[0])

Retrieved: ['RAG combines retrieval and generation for better answers.', 'Vector databases store embeddings for fast search.']
Distances: [0.81349945 1.9098997 ]


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = AutoModelForCausalLM.from_pretrained(model_name)

In [9]:
import torch
llm.to("cuda")

NameError: name 'llm' is not defined

In [ ]:
def rag_query(query):
    # Retrieve from FAISS
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k=2)
    context = "\n".join([documents[i] for i in indices[0]])

    # Prepare prompt
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate response
    outputs = llm.generate(
        inputs["input_ids"],
        max_new_tokens=50,
        do_sample=True,
        temperature=0.7
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer[len(prompt):].strip()

# Test it
query = "What is RAG?"
print(rag_query(query))

In [ ]:
print(rag_query("Why use vector databases?"))

In [ ]:
query = "What is RAG?"
query_embedding = model.encode([query])
distances, indices = index.search(query_embedding, k=2)
context = "\n".join([documents[i] for i in indices[0]])
answer = rag_query(query)
print(f"Context:\n{context}\n\nAnswer:\n{answer}")

In [ ]:
%time print(rag_query("What is RAG?"))

In [10]:
# Install dependencies (run once)
!pip install faiss-cpu sentence-transformers transformers torch numpy pandas -q

# Import libraries
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Prepare data
documents = [
    "RAG combines retrieval and generation for better answers.",
    "Vector databases store embeddings for fast search.",
    "Open-source models are free and powerful."
]

# Generate embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(documents, show_progress_bar=True)

# Set up FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Load language model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = AutoModelForCausalLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
llm.to(device)
print(f"Model loaded on: {device}")

# Define RAG function
def rag_query(query):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k=2)
    context = "\n".join([documents[i] for i in indices[0]])
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = llm.generate(
        inputs["input_ids"],
        max_new_tokens=50,
        do_sample=True,
        temperature=0.7
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer[len(prompt):].strip()

# Test it
query = "What is RAG?"
print(rag_query(query))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded on: cpu
RAG stands for Retrieval and Generation. It is a technique for combining retrieval and generation for better answers. RAG combines retrieval and generation by using a vector database to store embeddings for fast search. This means that when a
